In [2]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from parallel_descriptions import send_message_to_gemini_async
from km_utils import get_ToCs
from coap_prompts import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pipeline

In [3]:
ToCs = get_ToCs(base_path = 'coap_map')

ToC_sections = ToCs['ToC_sections']
ToC_chapters = ToCs['ToC_chapters']
ToC_articles_short_descriptions = ToCs['ToC_articles_short_descriptions']
ToC_articles_descriptions = ToCs['ToC_articles_descriptions']

In [7]:
user_query_ori = "что будет за дискредитацию армии рф?"
user_query = await get_perefrased_query(user_query_ori)
user_query = user_query['text_response']
user_query

'["Каковы последствия дискредитации Вооруженных сил Российской Федерации?", "Какие конкретные действия считаются дискредитацией?", "Какие меры наказания предусмотрены за дискредитацию?"]'

In [87]:
from datetime import datetime, timedelta
import google.generativeai as genai
import json, asyncio
from parallel_descriptions import RateLimiter

def split_dict_by_size(input_dict, size_limit=30000):
    """
    Splits a dictionary into multiple dictionaries, each as in a string equivalent not exceeding size_limit characters.
    """
    parts = []  # List to hold the resulting dictionaries
    current_part = {}  # Current dictionary part being filled
    current_size = 2  # Account for the empty dictionary braces '{}'

    for key, value in input_dict.items():
        # Estimate size of current item. Add 4 for ': ' and ', ' (the latter for all but the last item)
        item_size = len(repr(key)) + len(repr(value)) + 4
        if current_size + item_size > size_limit:
            # Add the current part to the list and start a new one
            parts.append(current_part)
            current_part = {key: value}
            current_size = 2 + item_size  # Reset size for the new part, including braces
        else:
            # Add item to the current part
            current_part[key] = value
            current_size += item_size

    # Add the final part if not empty
    if current_part:
        parts.append(current_part)

    return parts



async def get_chapter_navigation(user_query, ToC_chapters_chunks, rate_limiter):
    tasks = []
    for ToC_chunk in ToC_chapters_chunks:
        if isinstance(ToC_chunk, dict): 
            ToC_chunk = json.dumps(ToC_chunk, ensure_ascii=False)

        # ToC_chunk = json.dumps(ToC_chunk, ensure_ascii=False, indent=2)
        prompt = """ТЫ — AI-помощник, который для ответа на вопрос помогает находить главу и статью из КОАП РФ, чтобы более точно ответить на вопрос.
Тебе нужно выбрать несколько статей в которых может содержаться ответ или часть ответа. Выводи все потенциально полезные статьи.
Вот список глав со статьями:
<ToC_chapters>
{ToC_chapters}
</ToC_chapters>
Вот вопрос от юзера:
<query>{USER_QUERY}</query>
Выведи список с полнымми релевантных глав в порядке убывания важности с полным посимвольным совпадением как в приведеном списке <ToC_chapters>.
Формат вывода: ["Название главы 1", "Название главы 2"]
""".format(USER_QUERY=user_query, ToC_chapters=ToC_chunk)

        task = asyncio.create_task(send_message_to_gemini_async(user_input=prompt, rate_limiter=rate_limiter))
        tasks.append(task)
    
    descriptions = await asyncio.gather(*tasks)
    return descriptions

class RateLimiter:
    def __init__(self, calls_per_period, period=1.0):
        self.calls_per_period = calls_per_period
        self.period = timedelta(seconds=period)
        self.calls = []

    async def wait(self):
        now = datetime.now()
        
        while self.calls and now - self.calls[0] > self.period:
            self.calls.pop(0)
            
        if len(self.calls) >= self.calls_per_period:
            sleep_time = (self.period - (now - self.calls[0])).total_seconds()
            await asyncio.sleep(sleep_time)
            return await self.wait()

        self.calls.append(datetime.now())


rate_limiter = RateLimiter(5, 6)
section = 'Раздел II. Особенная часть'
ToC_chapters_chunks = split_dict_by_size(ToC_chapters[section], 30000)
await get_chapter_navigation(user_query, ToC_chapters_chunks, rate_limiter)


[{'text_response': '["Глава 20. Административные правонарушения, посягающие наобщественный порядок и общественную безопасность"]',
  'input_tokens': total_tokens: 5472,
  'output_tokens': total_tokens: 25},
 {'text_response': '["Глава 5. Административные правонарушения, посягающие на праваграждан"]',
  'input_tokens': total_tokens: 7423,
  'output_tokens': total_tokens: 21},
 {'text_response': '["Глава 13. Административные правонарушения в области связи иинформации"]',
  'input_tokens': total_tokens: 6614,
  'output_tokens': total_tokens: 19},
 {'text_response': 'Этот вопрос не связан с темами, перечисленными в списке глав.',
  'input_tokens': total_tokens: 6506,
  'output_tokens': total_tokens: 16},
 {'text_response': '["Глава 19. Административные правонарушения против порядка управления"]',
  'input_tokens': total_tokens: 3261,
  'output_tokens': total_tokens: 15}]

In [89]:
async def get_section_nagigation(USER_QUERY, ToC_sections):
    if isinstance(ToC_sections, dict):
        ToC_sections = json.dumps(ToC_sections, ensure_ascii=False, indent=2)

    prompt = """Вы — AI-помощник навигации, который помогает по вопросу от юзера находить пути к необходимой информации в КОАП РФ, чтобы точно ответить на вопрос.
Вам нужно выбрать 1 или несколько разделов первого уровня.
Вот список разделов с описаниями в формате json:
<map>
{ToC_sections}
</map>
Вот вопрос от юзера:
<query>{USER_QUERY}</query>
Выведите список JSON с путями навигации к соответствующей информации. Выводи только разделы.
Формат должен выглядеть так:
["Раздел 1", "Раздел 2"]
""".format(USER_QUERY=USER_QUERY, ToC_sections=ToC_sections)
    description = await send_message_to_gemini_async(prompt, attempt=1, max_attempts=10)
    return description


# async def get_chapter_nagigation(USER_QUERY, ToC_chapters):
#     if isinstance(ToC_chapters, dict):
#         ToC_chapters = json.dumps(ToC_chapters, ensure_ascii=False, indent=2)

#     prompt = """ТЫ — AI-помощник, который для ответа на вопрос помогает находить главу из КОАП РФ, чтобы более точно ответить на вопрос.
# Тебе нужно выбрать 1 или несколько глав в которых может содержаться ответ или часть ответа. Выводи все потенциально полезные главы, внутри глав будет содержатся много статей, некоторые из которых окажутся полезными.
# Вот список глав:
# <ToC_chapters>
# {ToC_chapters}
# </ToC_chapters>
# Вот вопрос от юзера:
# <query>{USER_QUERY}</query>
# Выведите список с полнымми названиями глав с полным посимвольным совпадением как в приведеном списке <ToC_chapters>.
# Формат должен выглядеть так:
# ["Глава 1", "Глава 2"]
# """.format(USER_QUERY=USER_QUERY, ToC_chapters=ToC_chapters)
#     description = await send_message_to_gemini_async(prompt, attempt=1, max_attempts=10)
#     return description


async def get_article_nagigation(USER_QUERY, articles):
    if isinstance(articles, dict):
        articles = json.dumps(articles, ensure_ascii=False, indent=2)

    prompt = """Вы — AI-помощник навигации, который помогает по вопросу от юзера находить пути к необходимой информации  в КОАП РФ, чтобы точно ответить на вопрос.
Вам нужно выбрать 1 или несколько статей. Старайся вывести все потенциально релевантные статьи.
Вот список статей с описаниями в формате json:
<map>
{articles}
</map>
Вот вопрос от юзера:
<query>{USER_QUERY}</query>
Выведите список JSON с путями навигации к соответствующей информации. Выводи полное название статьей.
Формат должен выглядеть так:
["Статья 1", "Статья 2"]
""".format(USER_QUERY=USER_QUERY, articles=articles)
    description = await send_message_to_gemini_async(prompt, attempt=1, max_attempts=10)
    return description


articles = []

# Chooes Sections
get_section_response = await get_section_nagigation(user_query, ToC_sections)
try: choosed_sections = ast.literal_eval(get_section_response['text_response'])
except: print(f'Section error: {get_section_response["text_response"]}')
# Chooes Chapters
for section in choosed_sections:
    print(section)
    # get_chapter_response = await get_chapter_nagigation(user_query, ToC_chapters[section])
    ToC_chapters_chunks = split_dict_by_size(ToC_chapters[section], 30000)
    descriptions = await get_chapter_navigation(user_query, ToC_chapters_chunks, rate_limiter)
    choosed_chapters = []
    for d in descriptions:
        try: text_response = ast.literal_eval(d['text_response'])
        except: print(f'Chapter error: {d["text_response"]}')
        if len(text_response) > 0:
            choosed_chapters.extend(text_response)
    
    # Chooes Articles
    for chapter in choosed_chapters:
        print(chapter)
        get_article_response = await get_article_nagigation(user_query, ToC_articles_short_descriptions[section][chapter])
        try: choosed_articles = ast.literal_eval(get_article_response['text_response'])
        except: print(f'Chapter error: {get_article_response["text_response"]}')

        for article in choosed_articles:
            print(article)
            article_text = nested_json['childs'][section]['childs'][chapter]['childs'][article]['content']
            articles.append({'section':section, 'chapter':chapter, 'article':article, 'article_text':article_text})

articles

Раздел II. Особенная часть
Chapter error: Этот вопрос не связан с темами, перечисленными в списке глав.
Глава 20. Административные правонарушения, посягающие наобщественный порядок и общественную безопасность
Статья 20.3.3. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской Федерации и ее граждан, поддержания международного мира и безопасности или исполнения государствен
Глава 5. Административные правонарушения, посягающие на праваграждан
Chapter error: В предоставленном тексте нет статей, касающихся публичных высказываний, которые могут быть расценены как дискредитация Вооруженных сил Российской Федерации.
Статья 20.3.3. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской Федерации и ее граждан, поддержания международного мира и безопасности или исполнения государствен


KeyError: 'Статья 20.3.3. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской Федерации и ее граждан, поддержания международного мира и безопасности или исполнения государствен'

In [81]:
async def get_final_response(USER_QUERY, context):
    prompt = """Ты профессиональный Юрист который отвечает на вопросы людей ориентируясь на статьи из КоАП РФ
Вот релевантная статья:
<article>
{context}
</article>
Вот вопрос:
<query>{USER_QUERY}</query>

Ответь нв вопрос, процитировав название статьи, раздел и главу. 
Если тебе не дали релевантную статью, попробуй ответить на вопрос, но скажи что "релевантная статья не найдена, попробуйте переформулировать вопрос".
Формат ответа:
<краткий ответ на вопрос пользователя>
<точный адрес статьи, включающий в себя название раздела, главы и название статьи>
""".format(USER_QUERY=USER_QUERY, context=context)
    description = await send_message_to_gemini_async(prompt, attempt=1, max_attempts=10)
    return description, prompt

get_article_response, prompt = await get_final_response(user_query, articles)
print(get_article_response['text_response'])

Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации влекут наложение административного штрафа.
Раздел II. Особенная часть, Глава 20. Административные правонарушения, посягающие наобщественный порядок и общественную безопасность, Статья 20.3.3. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской Федерации и ее граждан, поддержания международного мира и безопасности или исполнения государствен


Штраф за пересечение стоп-линии при красном сигнале светофора составляет 800 рублей.
Раздел II. Особенная часть, Глава 12. Административные правонарушения в области дорожного движения, Статья 12.12. Проезд на запрещающий сигнал светофора или на запрещающий жест регулировщика


In [86]:
from pprint import pprint
pprint(prompt)

('Ты профессиональный Юрист который отвечает на вопросы людей ориентируясь на '
 'статьи из КоАП РФ\n'
 'Вот релевантная статья:\n'
 '<article>\n'
 'Раздел II. Особенная часть\n'
 'Глава 12. Административные правонарушения в области дорожного движения\n'
 'Статья 12.12. Проезд на запрещающий сигнал светофора или на запрещающий жест '
 'регулировщика\n'
 '</article>\n'
 'Вот вопрос:\n'
 '<query>какой штраф за пересечении стоп-линии красный сигнал '
 'светофора?</query>\n'
 '\n'
 'Ответь нв вопрос, процитировав название статьи, раздел и главу. \n'
 'Если тебе не дали релевантную статью, попробуй ответить на вопрос, но скажи '
 'что "релевантная статья не найдена, попробуйте переформулировать вопрос".\n')


In [76]:
choosed_article[0]

'Статья 12.12. Проезд на запрещающий сигнал светофора или на запрещающий жест регулировщика'

'1. Проезд на запрещающий сигнал светофора или на запрещающий жест регулировщика, за исключением случаев, предусмотренных частью\xa01 статьи\xa012.10 настоящего Кодекса и частью\xa02 настоящей статьи, -\nвлечет наложение административного штрафа в размере одной тысячи рублей.\n2. Невыполнение требования Правил дорожного движения об остановке перед стоп-линией, обозначенной дорожными знаками или разметкой проезжей части дороги, при запрещающем сигнале светофора или запрещающем жесте регулировщика -\nвлечет наложение административного штрафа в размере восьмисот рублей.\n3. Повторное совершение административного правонарушения, предусмотренного частью 1 настоящей статьи, -\nвлечет наложение административного штрафа в размере пяти тысяч рублей или лишение права управления транспортными средствами на срок от четырех до шести месяцев.\n\n'

{'Статья 12.29. Нарушение Правил дорожного движения пешеходом или иным лицом, участвующим в процессе дорожного движения': 'Если ты пешеход или пассажир и нарушаешь правила дорожного движения, то можешь получить штраф в размере 500 рублей. Велосипедисты и другие участники дорожного движения могут получить штраф в 800 рублей, а если они находятся в состоянии опьянения, штраф будет от 1000 до 1500 рублей.',
 'Статья 12.7. Управление транспортным средством водителем, не имеющим права управления транспортным средством': 'Если ты сядешь за руль без водительских прав, тебе грозит штраф до 15 000 рублей. Если у тебя уже отняли права, и тебя снова поймали за рулем, штраф будет 30 000 рублей, или тебя могут посадить на 15 суток, или заставить работать без оплаты в течение 100-200 часов. И если тебя повторно поймают за рулем без прав, штраф может достигать 100 000 рублей или тебя могут обязать отработать до 200 часов.',
 'Статья 12.26. Невыполнение водителем транспортного средства требования о пр